In [2]:
!pip install langchain PyMuPDF faiss-cpu sentence-transformers transformers langchain-community

In [3]:
import fitz
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [4]:
def extract_from_pdf(pdf_file):
  doc = fitz.open(pdf_file)
  text = ""
  for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    text += page.get_text()
  return text

In [5]:
pdf_file="./tnfd.pdf"
document_text = extract_from_pdf(pdf_file)

In [6]:
print(len(document_text))

178505


In [7]:
embedding_model = HuggingFaceEmbeddings(model_name="t5-small")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [8]:
def chunk_text(text, chunk_size=1000, overlap=100):
  chunks = []
  start = 0
  while start <len(text):
    end = start + chunk_size
    if end > len(text):
      end = len(text)
    chunks.append(text[start:end])
    start += chunk_size - overlap
  return chunks

In [9]:
chunks = chunk_text(document_text)

In [10]:
chunks

['1\nv0.1 Beta Release – for consultation\nThe TNFD Nature-related Risk \n& Opportunity Management \nand Disclosure Framework\nBeta v0.1 Release\nA Prototype for Consultation \nwith Market Participants\nMarch 2022\n2\n3\nv0.1 Beta Release – for consultation\nv0.1 Beta Release – for consultation\nForeword\nWith half of the world’s GDP moderately or highly dependent on nature and its services, financial \ninstitutions and businesses can no longer afford to overlook nature in strategy, risk management \nand capital allocation decision making.\nThe size and frequency of nature-related losses, and related impact on economies and societies \nis becoming clearer every year. Encouragingly, so too is understanding that nature-related risk \nis as great a challenge as climate risk. As such, the release of this first beta version of the TNFD \nframework could not be more timely.\nSince the official launch of the Taskforce on Nature-related Financial Disclosures in June 2021, \nwe have seen the mo

In [11]:
vector_store = FAISS.from_texts(chunks, embedding_model)

In [12]:
retriever = vector_store.as_retriever()

In [13]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

In [17]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForQuestionAnswering.from_pretrained("t5-small")

Some weights of T5ForQuestionAnswering were not initialized from the model checkpoint at t5-small and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
hf_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer, max_new_tokens=200)

In [20]:
llm = HuggingFacePipeline(pipeline=hf_pipeline)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [21]:
rag_pipeline = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [27]:
def query_rag(question):
  response = rag_pipeline(question, return_only_outputs=True)
  return response

In [28]:
question = "What is the report about?"
response = query_rag(question)
print(response)

ValueError: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.


ecosystem services do we 
have a dependency or impact 
on at each priority location?
What are the 
corresponding risks for our 
organisation?
What strategy and resource 
allocation decisions should 
be made as a result of this 
analysis?
How will we set targets 
and define and measure 
progress?
What will we disclose in line 
with the TNFD disclosure 
recommendations?
Where and how do we 
present our nature-related 
disclosures?
Which biomes and 
ecosystems do these 
activities interface with?
What is the current 
integrity and importance 
of the ecosystems at each 
location?  
What are our nature-
related dependencies 
and impacts across our 
business at each priority 
location?
What existing risk 
mitigation and 
management approaches 
are we already applying?
At which locations is our 
organisation interfacin

In [ ]:
|